In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd

import random
import os

In [ ]:
# Label_Path = '/kaggle/input/signdata/cat_data_npy_word/Label_v2.csv'
# Label = pd.read_csv(Label_Path)

# Label = Label.sample(frac=1).reset_index(drop=True)

# Train_Label = Label[:2400]
# Val_Label = Label[2400:]

# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# print(Train_Label.head().to_markdown())
# print(Val_Label.head().to_markdown())

In [2]:
## Sign Language Label Encoding

finger_incoder = {'안녕하세요' : 0, '감사합니다' : 1, '맞다' : 2, '아니다' : 3,
                     '좋다' : 4, '싫다' : 5, '있다' : 6, '없다' : 7, '괜찮다' : 8,
                     '아프다' : 9, '잘하다' : 10, '못하다' : 11, '나' : 12, '너' : 13,
                     '가족' : 14, '수어' : 15, '약속' : 16, '이름' : 17, 
                     '머리' : 18, '목' : 19, '가렵다' : 20}

finger_decoder = {0 : '안녕하세요', 1 : '감사합니다', 2 : '맞다', 3  : "아니다",
                     4 : '좋다', 5 : '싫다', 6 : '있다', 7 : '없다', 8 : '괜찮다',
                     9 : '아프다', 10 : '잘하다', 11 : '못하다', 12 : '나', 13 : '너',
                     14 : '가족', 15 : '수어', 16 : '약속', 17 : '이름',
                     18 : '머리', 19 : '목', 20 : '가렵다'}

In [18]:
## Train, Validation DataSet Calss

# class Train_SignDataSet(Dataset):
#     def __init__(self, df_label):
#         super().__init__()

#         self.df_label = df_label

#     def __len__(self):
#         return len(self.df_label)
    
#     def __getitem__(self, idx):
#         data = np.load(self.df_label.iloc[idx]['path'])
#         data = self.DataTransform(data)
#         data = self.DataNormalization(data)
#         data = torch.tensor(data).type(torch.float32)

#         label = self.df_label.iloc[idx]['label']
#         label = finger_incoder[label]

#         return data, label
    
#     def DataNormalization(self, data):
#         data = data.reshape(30, 42, 3)

#         left_x = data[:, :21, 0]
#         right_x = data[:, 21:42, 0]
#         left_y = data[:, :21, 1]
#         right_y = data[:, 21:42, 1]
#         left_z = data[:, :21, 2]
#         right_z = data[:, 21:42, 2]

#         data[:, :21, 0] -= np.min(left_x)
#         data[:, :21, 0] /= (np.max(left_x) - np.min(left_x))
#         data[:, 21:42, 0] -= np.min(right_x)
#         data[:, 21:42, 0] /= (np.max(right_x) - np.min(right_x))

#         data[:, :21, 1] -= np.min(left_y)
#         data[:, :21, 1] /= (np.max(left_y) - np.min(left_y))
#         data[:, 21:42, 1] -= np.min(right_y)
#         data[:, 21:42, 1] /= (np.max(right_y) - np.min(right_y))

#         data[:, :21, 2] -= np.min(left_z)
#         data[:, :21, 2] /= (np.max(left_z) - np.min(left_z))
#         data[:, 21:42, 2] -= np.min(right_z)
#         data[:, 21:42, 2] /= (np.max(right_z) - np.min(right_z))
        
#         data = data.reshape(3780)
        
#         return data
    
#     def DataTransform(self, data):
#         x_points_random = np.random.uniform(-0.01, 0.01)
#         y_points_random = np.random.uniform(-0.01, 0.01)

#         data = data.reshape(30, 42, 3)

#         data[:, :21, 0] += x_points_random
#         data[:, 21:42, 0] += x_points_random
#         data[:, :21, 1] += y_points_random
#         data[:, 21:42, 1] += y_points_random
#         data = data.reshape(3780)

#         return data
    
# class Val_SignDataSet(Dataset):
#     def __init__(self, df_label):
#         super().__init__()
        
#         self.df_label = df_label
        
#     def __len__(self):
#         return len(self.df_label)
    
#     def __getitem__(self, idx):
#         data = np.load(self.df_label.iloc[idx]['path'])
#         data = self.DataNormalization(data)
#         data = torch.tensor(data).type(torch.float32)
        
#         label = self.df_label.iloc[idx]['label']
#         label = finger_incoder[label]
        
#         return data, label
    
#     def DataNormalization(self, data):
#         data = data.reshape(30, 42, 3)

#         left_x = data[:, :21, 0]
#         right_x = data[:, 21:42, 0]
#         left_y = data[:, :21, 1]
#         right_y = data[:, 21:42, 1]
#         left_z = data[:, :21, 2]
#         right_z = data[:, 21:42, 2]

#         data[:, :21, 0] -= np.min(left_x)
#         data[:, :21, 0] /= (np.max(left_x) - np.min(left_x))
#         data[:, 21:42, 0] -= np.min(right_x)
#         data[:, 21:42, 0] /= (np.max(right_x) - np.min(right_x))

#         data[:, :21, 1] -= np.min(left_y)
#         data[:, :21, 1] /= (np.max(left_y) - np.min(left_y))
#         data[:, 21:42, 1] -= np.min(right_y)
#         data[:, 21:42, 1] /= (np.max(right_y) - np.min(right_y))

#         data[:, :21, 2] -= np.min(left_z)
#         data[:, :21, 2] /= (np.max(left_z) - np.min(left_z))
#         data[:, 21:42, 2] -= np.min(right_z)
#         data[:, 21:42, 2] /= (np.max(right_z) - np.min(right_z))
        
#         data = data.reshape(3780)
        
#         return data

In [3]:
## DataSet Class for WebCam Data

class Cam_SignDataSet(Dataset):
    def __init__(self, df_label):
        super().__init__()
        
        self.df_label = df_label
        
    def __len__(self):
        return len(self.df_label)
    
    def __getitem__(self, idx):
        data = self.df_label
        data = self.DataNormalization(data)
        data = torch.tensor(data).type(torch.float32)
        
        return data
    
    def DataNormalization(self, data):
        data = data.reshape(30, 42, 3)

        left_x = data[:, :21, 0]
        right_x = data[:, 21:42, 0]
        left_y = data[:, :21, 1]
        right_y = data[:, 21:42, 1]
        left_z = data[:, :21, 2]
        right_z = data[:, 21:42, 2]

        data[:, :21, 0] -= np.min(left_x)
        data[:, :21, 0] /= (np.max(left_x) - np.min(left_x))
        data[:, 21:42, 0] -= np.min(right_x)
        data[:, 21:42, 0] /= (np.max(right_x) - np.min(right_x))

        data[:, :21, 1] -= np.min(left_y)
        data[:, :21, 1] /= (np.max(left_y) - np.min(left_y))
        data[:, 21:42, 1] -= np.min(right_y)
        data[:, 21:42, 1] /= (np.max(right_y) - np.min(right_y))

        data[:, :21, 2] -= np.min(left_z)
        data[:, :21, 2] /= (np.max(left_z) - np.min(left_z))
        data[:, 21:42, 2] -= np.min(right_z)
        data[:, 21:42, 2] /= (np.max(right_z) - np.min(right_z))
        
        data = data.reshape(3780)
        
        return data

In [19]:
# train_ds = Train_SignDataSet(Train_Label)
# val_ds = Val_SignDataSet(Val_Label)

# train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, pin_memory=True)
# val_dl = DataLoader(val_ds, batch_size=32, shuffle=False, pin_memory=True)

In [20]:
# train_ds[0][0].shape

torch.Size([3780])

In [4]:
def save_model(name, model):
    torch.save(model.state_dict(), f'{name}.pth')
    
def load_model(model, name, path='.'):
    data = torch.load(os.path.join(path, f'{name}.pth'), map_location='cpu')
    model.load_state_dict(data)
    return model

In [5]:
class ASLModel(nn.Module):
    def __init__(self):
        super(ASLModel, self).__init__()
        self.layer0 = nn.Linear(3780, 1890)
        self.layer1 = nn.Linear(1890, 945)
        self.layer2 = nn.Linear(945, 21)
        
    def forward(self, x):
        x = self.layer0(x)
        x = F.relu(x)
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        return x

In [ ]:
## Model Train

# import time

# def train_model(train_dl, train_loss_list, val_loss_list, val_acc_list, time_list, epochs):
#     train_total = len(train_dl.dataset)
#     val_total = len(val_dl.dataset)
#     save_loss = 10
    
#     model = ASLModel()
#     model = model.to(DEVICE)
    
#     loss_fun = nn.CrossEntropyLoss()
#     optim = torch.optim.Adam(model.parameters(), lr=0.00001)
#     scheduler = torch.optim.lr_scheduler.OneCycleLR(optim, max_lr=0.0005, epochs=epochs, steps_per_epoch=len(train_dl))
    
#     for epoch in tqdm(range(epochs), desc='Epoch'):
#         start = time.time()
#         model.train()
#         train_loss = 0
#         train_correct = 0
        
#         val_loss = 0
#         val_correct = 0
        
#         with tqdm(train_dl, desc='Train') as train_progress:
#             for batch_idx, (X, y) in enumerate(train_progress):
#                 model.train()
#                 optim.zero_grad()
#                 y = y.to(DEVICE)
# #                 print(y)
                
#                 y_pred = model(X.to(DEVICE))
#                 y_loss = loss_fun(y_pred, y)
                
#                 train_loss += y_loss.item() * y.size(0)
                
#                 y_loss.backward()
#                 optim.step()
#                 scheduler.step()
                
#                 predict = torch.argmax(y_pred, dim=-1)
#                 train_correct += y.eq(predict).sum()
                
#             train_acc = train_correct / train_total
#             train_loss = train_loss / train_total
            
#             train_loss_list.append(train_loss)
            
#             print('epoch: {} --> train_loss: {:.4f} - train_acc: {:.4f} - '.format(epoch, train_loss, train_acc), end='')
            
#         with tqdm(val_dl, desc='Val') as val_progress:
#             for batch_idx, (X, y) in enumerate(val_progress):
#                 model.eval()
#                 y = y.to(DEVICE)
                
#                 y_pred = model(X.to(DEVICE))
#                 y_loss = loss_fun(y_pred, y)
                
#                 val_loss += y_loss.item() * y.size(0)
#                 predict = torch.argmax(y_pred, dim=-1)
#                 val_correct += y.eq(predict).sum()
                
#             val_acc = val_correct / val_total
#             val_loss = val_loss / val_total
            
#             val_loss_list.append(val_loss)
#             val_acc_list.append(val_acc.item())
            
#             if val_loss < save_loss:
#                 save_loss = val_loss
#                 save_model('Save_Model', model)
#                 print("Save_Model!")
            
#             print('epoch: {} --> val_loss: {:.4f} - val_acc: {:.4f}'.format(epoch, val_loss, val_acc))
            
#         end = time.time()
#         elapsed_time = end - start
#         time_list.append(elapsed_time)
    
# train_loss_list = []
# val_loss_list = []
# val_acc_list = []
# time_list = []

# train_model(train_dl, train_loss_list, val_loss_list, val_acc_list, time_list, 10)

In [13]:
## Real Time WebCam Data Operation & UI

import cv2
import time
from PIL import ImageFont, ImageDraw, Image
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

new_model = load_model(ASLModel(), 'Save_Model')

count = 0

t_result = '0'
right_keypoints = []
left_keypoints = []
both_hands_list = []
t_pred_list = []
result_keypoint = []
result_word_lst = []
text = ''
starting_delay_count = 0

cap = cv2.VideoCapture(cv2.CAP_DSHOW + 0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 540)
a1 = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
a2 = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

print(a1, a2)
# cap.set(cv2.CAP_PROP_FPS, 30)
frame_rate = 30
prev = 0
result_str = ''
back_img1 = np.zeros((100, 1024, 3))
back_img1[:] = 255
        
cap_fps = cap.get(cv2.CAP_PROP_FPS)
cap_fps_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
cap_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
cap_pos_frames = cap.get(cv2.CAP_PROP_POS_FRAMES)

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("카메라를 찾을 수 없습니다.")
            break
        
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        
        right_keypoint = np.zeros((21, 3))
        left_keypoint = np.zeros((21, 3))
        
        try:
            if len(results.multi_hand_landmarks) == 2:
                for i in range(21):
                    if results.multi_hand_landmarks[0].landmark[0].x > results.multi_hand_landmarks[1].landmark[0].x:
                        right_keypoint[i, 0] = results.multi_hand_landmarks[0].landmark[i].x
                        right_keypoint[i, 1] = results.multi_hand_landmarks[0].landmark[i].y
                        right_keypoint[i, 2] = results.multi_hand_landmarks[0].landmark[i].z

                        left_keypoint[i, 0] = results.multi_hand_landmarks[1].landmark[i].x
                        left_keypoint[i, 1] = results.multi_hand_landmarks[1].landmark[i].y
                        left_keypoint[i, 2] = results.multi_hand_landmarks[1].landmark[i].z
                    else:
                        right_keypoint[i, 0] = results.multi_hand_landmarks[1].landmark[i].x
                        right_keypoint[i, 1] = results.multi_hand_landmarks[1].landmark[i].y
                        right_keypoint[i, 2] = results.multi_hand_landmarks[1].landmark[i].z

                        left_keypoint[i, 0] = results.multi_hand_landmarks[0].landmark[i].x
                        left_keypoint[i, 1] = results.multi_hand_landmarks[0].landmark[i].y
                        left_keypoint[i, 2] = results.multi_hand_landmarks[0].landmark[i].z
                both_hand = np.concatenate((right_keypoint, left_keypoint))
        except:
            pass
        
        
        count += 1
        
        if count > 29 and results.multi_hand_landmarks is not None:
            try:
                result_keypoint = []
                for i in range(30):
                    result_keypoint.append(both_hand)
                save_file = np.array(result_keypoint)
                save_file = save_file.reshape(3780)
                save_file = pd.DataFrame(save_file)
                save_file = save_file.fillna(0)
                save_file = save_file.replace(0, np.NaN)
                save_file = save_file.fillna(method='ffill')
                save_file = save_file.to_numpy()
                s_data = Cam_SignDataSet(save_file)
                t_pred = new_model(s_data[0])
                t_predict = torch.argmax(t_pred, dim=-1)
                t_pred_list.append(t_predict.item())
                t_result = t_predict
                result_str = str(finger_decoder[int(t_result)])
                result_word_lst.append(result_str)
                result_keypoint = []
                count = 0
            except:
                result_keypoint = []
                count = 0
                continue

        # 이미지에 손 주석을 그립니다.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.DrawingSpec(
                        color = (0,255,0),
                        thickness = 1,
                        circle_radius = 3),
                    mp_drawing_styles.DrawingSpec(
                        color = (255, 255, 255),
                        thickness = 2,
                        circle_radius = 1))
        
        if starting_delay_count < 30:
            result_word_lst = ['프로그램 시작 대기']
            starting_delay_count += 1
        elif starting_delay_count == 30:
            result_word_lst = ['준비 완료 / 시작']
            starting_delay_count += 1
        
        if cv2.waitKey(1) & 0xFF == ord('d'):
            result_word_lst = []
                
        image = cv2.flip(image, 1)
        
        org = (5, 20)
        font = ImageFont.truetype("fonts/gulim.ttc", 30)
        box_color = (255, 255, 255)
        font_scale = 2
        thinkness = 3
        
        back_img1 = np.zeros((100, 1024, 3))
        back_img1[:] = 255
        
        count_str = str(int(count))
        
        if len(text) > 40:
            del result_word_lst[0]
        
        words_sum = ""
        for i in result_word_lst:
            words_sum += i + ", "
        
        text = " 수어단어 :: " +  words_sum
        
        back_img1 = Image.fromarray(back_img1.astype(np.uint8))
        draw = ImageDraw.Draw(back_img1) 
        draw.text(org, text, font=font, fill=(0,0,0))
        
        back_img1 = np.array(back_img1)
        
        image = Image.fromarray(image.astype(np.uint8))
        count_font = ImageFont.truetype("fonts/gulim.ttc", 40)
        draw_count = ImageDraw.Draw(image)
        draw_count.text((5, 5), count_str, fill=(0, 255, 0), font=count_font)
        image = np.array(image)
        
        aa = np.concatenate((image, back_img1), axis=0)
        
        aa = np.array(aa, dtype=np.uint8)
        
        cv2.imshow('MediaPipe Hands', aa)
    
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

1024.0 576.0
